In [2]:
import geopandas as gpd
import pandas as pd
import folium

from sqlalchemy import create_engine
from secrets.con_str import con_str

import warnings
# warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore")


engine = create_engine(con_str)

In [3]:
sql = """
WITH divided_foots AS (
	SELECT foot_gid
	FROM intersections_reduced
	GROUP BY foot_gid
	HAVING COUNT(*) > 1
)
SELECT intersections_reduced.foot_gid
FROM intersections_reduced
INNER JOIN divided_foots
ON divided_foots.foot_gid = intersections_reduced.gid
WHERE area/foot_area < 0.1
AND area/parcel_area < 0.1
ORDER BY area DESC
LIMIT 20;
"""

messy_foot_gids = pd.read_sql(sql, engine)
messy_foot_gids.head()

,foot_gid
0,250899
1,573661
2,243737
3,434298
4,124982


In [4]:
foot_gid_list_str = ', '.join([str(foot_gid) for foot_gid in messy_foot_gids['foot_gid']])

sql = f"""
SELECT DISTINCT geom
FROM footprints
WHERE gid IN ({foot_gid_list_str})
"""

messy_footprints = gpd.read_postgis(sql, engine)
messy_footprints = messy_footprints.to_crs(epsg=4326)

sql = f"""
WITH parcel_gids AS (
  SELECT DISTINCT parcel_gid AS gid
  FROM intersections
  WHERE foot_gid IN ({foot_gid_list_str})
)
SELECT
  parcels.geom,
  array_agg(parcels.pin) AS parcel_pins
FROM parcels
INNER JOIN parcel_gids
ON parcel_gids.gid = parcels.gid
GROUP BY parcels.geom
"""

messy_parcels = gpd.read_postgis(sql, engine)
messy_parcels = messy_parcels.to_crs(epsg=4326)

print(
  len(messy_parcels),
  len(messy_footprints)
)

153 20


In [8]:
type(messy_parcels['parcel_pins'].iloc[0])

list

In [17]:
m = folium.Map(location=[40.4406, -79.9959], zoom_start=10, tiles='CartoDB positron')

# FOOTPRINTS
for _, r in messy_footprints.iterrows():
  # Without simplifying the representation of each borough,
  # the map might not be displayed
  sim_geo = gpd.GeoSeries(r['geom'])
  # .simplify(tolerance=0.001)
  geo_j = sim_geo.to_json()
  geo_j = folium.GeoJson(
    data=geo_j, style_function=lambda x: {
      'fillColor': 'rgba(255,0,0,0.5)',
      'color':     'rgba(255,0,0,0.8)'
    })
  # folium.Popup(r['BoroName']).add_to(geo_j)
  geo_j.add_to(m)

# PARCELS
for _, r in messy_parcels.iterrows():
  # Without simplifying the representation of each borough,
  # the map might not be displayed
  sim_geo = gpd.GeoSeries(r['geom'])
  # .simplify(tolerance=0.001)
  geo_j = sim_geo.to_json()
  geo_j = folium.GeoJson(
    data=geo_j, style_function=lambda x: {
      'fillColor': 'rgba(0,0,255,0.5)',
      'color':     'rgba(0,0,255,0.8)',
      'weight':    '0.5'
    })
  folium.Tooltip(str(r['parcel_pins'])).add_to(geo_j)
  geo_j.add_to(m)

m